In [ ]:
import requests
import snowflake.connector
import os
import snowflake.connector
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pprint

s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

CEDL_HOME = os.environ['CEDL_HOME']
nexus_connectionProfile = CEDL_HOME + '/etc/.sf.nexus.profile'
s3_connectionProfile = CEDL_HOME + '/etc/.s3_connection_profile'



def create_snowflake_connection():
    try:
        pathExist = os.path.exists(nexus_connectionProfile)
        if (not pathExist):
            print('The profile {} doesn''t exist'.format(nexus_connectionProfile))
            exit(1)
        profileFile = open(nexus_connectionProfile)
        for line in profileFile:
            if (line.split('=')[0] == 'snowflakeAccount'):
                snowflakeAccount = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeUsername'):
                snowflakeUsername = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakePassword'):
                snowflakePassword = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeRole'):
                snowflakeRole = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeDBName'):
                snowflakeDBName = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeWarehouse'):
                snowflakeWarehouse = line.split('=')[1].replace('\n', '')
            else:
                pass
        profileFile.close()
        if (len(snowflakeAccount) == 0 or len(snowflakeUsername) == 0 or len(snowflakePassword) == 0 or len(
                snowflakeRole) == 0 or len(snowflakeDBName) == 0 or len(snowflakeWarehouse) == 0):
            print('some parameters are missing from {}'.format(nexus_connectionProfile))
            exit(1)
        conn = snowflake.connector.connect(user=snowflakeUsername, password=snowflakePassword, account=snowflakeAccount,
                            warehouse=snowflakeWarehouse, database=snowflakeDBName)
        print("connected to SNOWFLAKE Database.")
    except snowflake.connector.Error as e:
        print('Error connecting to SNOWFLAKE Database - {}'.format(e))
        exit(1)
    return conn

def make_request(endpoint):
    base_url = 'https://jsonplaceholder.typicode.com'
    url = f"{base_url}/{endpoint}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

        # If the response is successful (status code 200)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def insert_into_snowflake(data, snowflake_config,connection):
    try:
        cursor = connection.cursor()

        # Assuming a hypothetical 'posts' table in Snowflake
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {snowflake_config['database']}.{snowflake_config['schema']}.posts (id INTEGER, title VARCHAR, body VARCHAR)")

        # Insert data into the 'posts' table
        cursor.executemany(f"INSERT INTO {snowflake_config['database']}.{snowflake_config['schema']}.posts VALUES (%s, %s, %s)", [(post['id'], post['title'], post['body']) for post in data])

        connection.commit()
        print("Data inserted into Snowflake successfully!")

    except snowflake.connector.Error as e:
        print(f"Snowflake Error: {e}")

    finally:
        cursor.close()
        connection.close()

if __name__ == "__main__":
    # Example: Get posts
    posts_data = make_request('posts')

    if posts_data:
        # Snowflake configuration
        snowflake_config = {
            'database': 'NEXUS',
            'schema': 'TEMP',
        }

        snowflake_connection = create_snowflake_connection()

        # Example: Insert data into Snowflake
        insert_into_snowflake(posts_data, snowflake_config, snowflake_connection)
    else:
        print("Failed to retrieve data from the API.")
